In [8]:
###############################################################################
#
# EDINETから特定の企業の有価証券報告書をXBRL形式でダウンロードし，項目を抜き出し，
# テキストに変換し，Dataframeに格納し，Excel形式で書き出すプログラム
#
# by Y.Maeda
# first created 2021/10/20
# modified      2022/05/06
#
#
###############################################################################

#　＜＜＜注意＞＞＞
# ファイル名のセットは適切に行ってください（フォルダー名などは各自によって異なる）
#
#
#　dir_path_dic = {'四半期報告書':r'EDINET_S/Quarter2022',
#　　　　　　　　　　　　　　　　　　　　　'有価証券報告書':r'EDINET_S/Report2022'}
#　pythonのプログラムの入っているフォルダーに，フォルダー「EDINET_S」を作成し，
#　その中にフォルダー「研究会配布用」を作成し，その中に「Quarter2022」とフォルダー「Report2022」を作成する．
#

In [9]:
%pwd

'C:\\Users\\shintaro\\Downloads'

In [10]:
from datetime import date,timedelta, datetime
import requests
import os
import zipfile
import subprocess
import json
import sys
import re

import numpy as np
import pandas as pd
from pandas import DataFrame
import matplotlib.pyplot as plt
import seaborn as sns; sns.set();
%matplotlib inline

from edinet_xbrl.edinet_xbrl_parser import EdinetXbrlParser
import glob
from bs4 import BeautifulSoup

import urllib3
from urllib3.exceptions import InsecureRequestWarning
urllib3.disable_warnings(InsecureRequestWarning)

#
#　「EDINET_S」という名前のフォルダーをプログラムの入っているフォルダーに作成しておく
#　「EDINET_S」の中に「Quarter2022」と「Report2022」名前のフォルダーを作成しておく

SSDPath='/EDINE_S'
QPath=SSDPath+'/Quarter2022'
RPath=SSDPath+'/Report2022'

In [11]:
def get_one_xbrl_data(path, key, context_ref):
    parser = EdinetXbrlParser()
 
    ## 指定したxbrlファイルをパースする
    xbrl_file_path = path
    edinet_xbrl_object = parser.parse_file(xbrl_file_path)
 
    ## データの取得
    data = edinet_xbrl_object.get_data_by_context_ref(key, context_ref)
    if data is not None:
        return data.get_value()    
    

def get_list(start,end,company_list):
    #指定した期間、報告書種類、会社で報告書を取得し、取得したファイルのパスの辞書を返す
    #
    #取得期間の日付リストを作成
    day_term = [start + timedelta(days=i) for i in range((end - start).days)]
    
    #データ抜出時に使用する、有価証券報告書および四半期報告書のコードの設定
    ordinance_code = "010"
    form_code_quart ="043000" # 四半期報告書
    form_code_securities ="030000" #有価証券報告書
    
    # EDINETのAPIで、書類一覧を取得し、各日ごとに必要な書類の項目を抜き出し
    quart_list =[] # 四半期報告書のリスト
    securities_list =[] # 四半期報告書のリスト

    print('EDINETへのアクセスを開始')
    for i,day in enumerate(day_term):
        url = "https://disclosure.edinet-fsa.go.jp/api/v1/documents.json"
        params = {"date": day, "type": 2}
        
        # 進捗表示
        if i % 50 == 0:
            print(f'{i}日目：{day}を開始')
        
        # EDINETから1日の書類一覧を取得
        res = requests.get(url, params=params, verify=False)
        
        # 必要な書類の項目を抜き出し
        if res.ok:
            json_data = res.json()
            if 'results' in json_data:
                for data in json_data['results']:
                    # 指定した会社の指定した書類を抜き出し
                    if data['ordinanceCode'] == ordinance_code \
                        and data['formCode'] == form_code_quart \
                        and data['filerName'].replace('株式会社', '') in company_list:
                            quart_list.append(data)
                            print(data['filerName'].replace('株式会社', ''))
                    elif data['ordinanceCode'] == ordinance_code \
                        and data['formCode'] == form_code_securities \
                        and data['filerName'].replace('株式会社', '') in company_list:
                            securities_list.append(data) 
                            print(data['filerName'].replace('株式会社', ''))
        else:
            print(f'アクセス失敗かも{day}')
    list_dic = {'四半期報告書':quart_list,'有価証券報告書':securities_list} 
    return list_dic


def get_zip(list_dic):
    #取得したいデータをzipファイルで取得してファイルパスのリストを返す
    dir_path_dic = {'四半期報告書':r'EDINET_S/Quarter2022',
                    '有価証券報告書':r'EDINET_S/Report2022'}
    # ダウンロードした有価証券報告書のパスを格納する辞書   
    file_path_dic = {'四半期報告書':[],'有価証券報告書':[]} 
    
    for key in list_dic.keys():
        for doc in list_dic[key]:
            #doc['filerName']
            FN=doc['filerName'].replace('株式会社', '')
            #print(doc)
            files = os.listdir(dir_path_dic[key]+'/'+FN)
            #print(dir_path_dic[key]+'/'+FN)
            #print('files:',files)
            # zipファイルパスのリストの作成
            file_name = doc['filerName'].replace('株式会社', '') + '_' + doc['docID']
            file_path =  os.path.join(dir_path_dic[key], FN+'/'+file_name + ".zip")
            file_path_dic[key].append([file_path,doc['docID'],FN])
            
            #print('file_name',file_name)
            #print('file_path',file_path)
            #print('file_name',file_name)
            # 所有していないファイルの場合はDLを行う
            if file_name+'.zip' not in files:
                # ファイルを取得
                print('ファイルを取得：',file_name)
                url_zip = "https://disclosure.edinet-fsa.go.jp/api/v1/documents/" + doc['docID']
                params_zip = {"type": 1}
                # データのDL
                res_zip = requests.get(url_zip, params=params_zip, verify=False, stream=True)
                # zipとして保存
                if res_zip.status_code == 200:
                    with open(file_path, 'wb') as f:
                        for chunk in res_zip.iter_content(chunk_size=1024):
                            if chunk:
                                f.write(chunk)
                                f.flush()
            else:
                print('ファイルは既にダウンロード済みです：',file_name)
    return file_path_dic


def zip_to_df(file_path_dic):
    #ダウンロードしたzipをdfに変換して各会社のdicにして返す
    all_df_dic = {}
    for key in file_path_dic.keys():
        print(f'{key}データの変換を開始')        
        df_dic = {}
        for i,company_zip in enumerate(file_path_dic[key]):
            # 進捗表示
            if i % 100 == 0:
                print(f'{i}ファイル目を開始')
            if key=='四半期報告書':
                zipPath='EDINET_S/Quarter2022/'+company_zip[2]+'/'+company_zip[1]
            else:
                zipPath='EDINET_S/Report2022/'+company_zip[2]+'/'+company_zip[1]
            if os.path.exists(zipPath):
                print('すでにZIP解凍済み：',company_zip[0],'zipPath：',zipPath)
            else:
                print('解凍中ZIP：',company_zip[0],'zipPath：',zipPath)
                with zipfile.ZipFile(company_zip[0]) as existing_zip:
                    existing_zip.extractall(zipPath)

def beep():
    os.system('afplay /System/Library/Sounds/Sosumi.aiff')

def beep3():
    for i in range(3):
        os.system('afplay /System/Library/Sounds/Sosumi.aiff')



In [12]:
##################################################################
#
#  ＜＜＜＜＜EDINETにアクセスして，Zipファイルをダウンロードする＞＞＞＞＞
#
##################################################################

#
# company_list：会社名
# n：n年前に遡ってダウンロードする
#

company_list={'サンリオ','イチカワ'}

# ここで年数を指定（n=1の場合は過去1年分）
n=1

#
# 会社のリストを読み込み
#

# 取得期間の設定：startDay日前からn日分（直近の場合はstartDay=0）
startDay = 0
delta_day = 365*n
end = date.today()-timedelta(days=startDay)
start = date.today()-timedelta(days=startDay)- timedelta(days=delta_day)

#
# フォルダーの作成（フォルダー名は各自設定する）
#
for i in company_list:
    # ダウンロードしたデータのフォルダパス
    try:
        os.mkdir('EDINET_S/Quarter2022/'+i)
    except OSError as e:
        print('フォルダーは既にあります：',i)
    try:
        os.mkdir('EDINET_S/Report2022/'+i)
    except OSError as e:
        print('フォルダーは既にあります：',i)

#　パスの設定    
# XBRLデータの取得 
list_dic = get_list(start,end,company_list)
file_path_dic = get_zip(list_dic)

#
# zipファイルの解凍
#
zip_to_df(file_path_dic)

#処理の終了
beep3()

フォルダーは既にあります： イチカワ
フォルダーは既にあります： イチカワ
フォルダーは既にあります： サンリオ
フォルダーは既にあります： サンリオ
EDINETへのアクセスを開始
0日目：2021-05-25を開始
サンリオ
イチカワ
50日目：2021-07-14を開始
イチカワ
サンリオ
100日目：2021-09-02を開始
150日目：2021-10-22を開始
サンリオ
イチカワ
200日目：2021-12-11を開始
250日目：2022-01-30を開始
サンリオ
イチカワ
300日目：2022-03-21を開始
350日目：2022-05-10を開始
ファイルは既にダウンロード済みです： イチカワ_S100M607
ファイルは既にダウンロード済みです： サンリオ_S100M9K7
ファイルは既にダウンロード済みです： サンリオ_S100MTFQ
ファイルは既にダウンロード済みです： イチカワ_S100MV37
ファイルは既にダウンロード済みです： サンリオ_S100NG3A
ファイルは既にダウンロード済みです： イチカワ_S100NGG2
ファイルは既にダウンロード済みです： サンリオ_S100LOB9
ファイルは既にダウンロード済みです： イチカワ_S100LUEG
四半期報告書データの変換を開始
0ファイル目を開始
すでにZIP解凍済み： EDINET_S/Quarter2022\イチカワ/イチカワ_S100M607.zip zipPath： EDINET_S/Quarter2022/イチカワ/S100M607
すでにZIP解凍済み： EDINET_S/Quarter2022\サンリオ/サンリオ_S100M9K7.zip zipPath： EDINET_S/Quarter2022/サンリオ/S100M9K7
すでにZIP解凍済み： EDINET_S/Quarter2022\サンリオ/サンリオ_S100MTFQ.zip zipPath： EDINET_S/Quarter2022/サンリオ/S100MTFQ
すでにZIP解凍済み： EDINET_S/Quarter2022\イチカワ/イチカワ_S100MV37.zip zipPath： EDINET_S/Quarter2022/イチカワ/S100MV37
すでにZIP解凍済み： EDINET_S/

In [13]:
#print(list_dic)

In [14]:
cd C:/Users/shintaro/Downloads/

C:\Users\shintaro\Downloads


In [15]:
####################################################
#
#  ＜＜＜書類の情報を格納したDataframeの作成＞＞＞
#
####################################################

df = pd.DataFrame(columns = ['会社名','書類名','日付','docID','docDescription','XBRLパス'])
df = df.set_index(['会社名','書類名','日付'])
#dir_path_dic={}

#for CName in company_list:

for key in list_dic.keys():
    print(key)
    DocType=key
    for doc in list_dic[DocType]:
        print('doc:',doc,'\n')
        CName = doc['filerName'].replace('株式会社', '')
        #file_name = CName + '_' + doc['docID']
        df.loc[(CName,DocType,doc['periodEnd']),'docID']=doc['docID']
        df.loc[(CName,DocType,doc['periodEnd']),'docDescription']=doc['docDescription']

        if DocType=='四半期報告書':
            dir_path='EDINET_S/Quarter2022/'+CName+'/'+doc['docID']
            df.loc[(CName,DocType,doc['periodEnd']),'XBRLパス']=dir_path
        else:
            dir_path='EDINET_S/Report2022/'+CName+'/'+doc['docID']
            df.loc[(CName,DocType,doc['periodEnd']),'XBRLパス']=dir_path

df = df.sort_index()
df.index.is_lexsorted()
#df

#
#  Dataframeを保存
#
fName='C:/Users/shintaro/Downloads/EDINET_S/FileList.csv'
df.to_csv(fName,encoding='utf_8_sig')
subprocess.Popen([r'C:/Program Files/Microsoft Office/root/Office16/EXCEL.EXE', r'C:/Users/shintaro/Downloads/EDINET_S/FileList.csv'])

四半期報告書
doc: {'seqNumber': 1467, 'docID': 'S100M607', 'edinetCode': 'E00572', 'secCode': '35130', 'JCN': '1010001000708', 'filerName': 'イチカワ株式会社', 'fundCode': None, 'ordinanceCode': '010', 'formCode': '043000', 'docTypeCode': '140', 'periodStart': '2021-04-01', 'periodEnd': '2021-06-30', 'submitDateTime': '2021-08-06 15:53', 'docDescription': '四半期報告書－第98期第1四半期(令和3年4月1日－令和3年6月30日)', 'issuerEdinetCode': None, 'subjectEdinetCode': None, 'subsidiaryEdinetCode': None, 'currentReportReason': None, 'parentDocID': None, 'opeDateTime': None, 'withdrawalStatus': '0', 'docInfoEditStatus': '0', 'disclosureStatus': '0', 'xbrlFlag': '1', 'pdfFlag': '1', 'attachDocFlag': '0', 'englishDocFlag': '0'} 

doc: {'seqNumber': 328, 'docID': 'S100M9K7', 'edinetCode': 'E02655', 'secCode': '81360', 'JCN': '5010701003956', 'filerName': '株式会社サンリオ', 'fundCode': None, 'ordinanceCode': '010', 'formCode': '043000', 'docTypeCode': '140', 'periodStart': '2021-04-01', 'periodEnd': '2021-06-30', 'submitDateTime': '2021-08

C:\Users\shintaro\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2901: PerformanceWarning: indexing past lexsort depth may impact performance.
  result = self._run_cell(
C:\Users\shintaro\AppData\Local\Temp/ipykernel_28268/1903087517.py:31: FutureWarning: MultiIndex.is_lexsorted is deprecated as a public function, users should use MultiIndex.is_monotonic_increasing instead.
  df.index.is_lexsorted()


<Popen: returncode: None args: ['C:/Program Files/Microsoft Office/root/Offi...>

In [16]:
cd /Users/shintaro/Downloads

C:\Users\shintaro\Downloads


In [17]:
#####################################################
#
#
# ＜＜＜＜＜EDINETの分析を行う会社と項目の設定＞＞＞＞＞
#
#
#####################################################

cNamelist=company_list

DocType='有価証券報告書'

#
# EDINETdataファイルの読み込み
#

fileName='EDINET_S/FileList.csv'
EDINETdata = pd.read_csv(fileName)
#EDINETdata = EDINETdata.set_index(['会社名','書類名','日付','docID','docDescription',	'XBRLパス'])
        
print('cNamelist：',cNamelist)

#
# データベースの抽出
#
EDINETdata2=EDINETdata.loc[EDINETdata['書類名'] == '有価証券報告書']

項目辞書={
    '研究開発':'jpcrp_cor:ResearchAndDevelopmentActivitiesTextBlock',
    '経営方針':'jpcrp_cor:BusinessPolicyBusinessEnvironmentIssuesToAddressEtcTextBlock',
    '事業リスク':'jpcrp_cor:BusinessRisksTextBlock',
    'ガバナンス':'jpcrp_cor:OverviewOfCorporateGovernanceTextBlock'
     }

cNamelist： {'イチカワ', 'サンリオ'}


In [18]:
EDINETdata2 = EDINETdata2.set_index(['会社名','書類名','日付'])

In [19]:
EDINETdata2

docID                    docDescription  \
会社名  書類名     日付                                                       
イチカワ 有価証券報告書 2021-03-31  S100LUEG  有価証券報告書－第97期(令和2年4月1日－令和3年3月31日)   
サンリオ 有価証券報告書 2021-03-31  S100LOB9  有価証券報告書－第61期(令和2年4月1日－令和3年3月31日)   

                                                    XBRLパス  
会社名  書類名     日付                                             
イチカワ 有価証券報告書 2021-03-31  EDINET_S/Report2022/イチカワ/S100LUEG  
サンリオ 有価証券報告書 2021-03-31  EDINET_S/Report2022/サンリオ/S100LOB9

In [20]:
辞書Keyリスト=list(項目辞書.keys())
辞書Keyリスト

#
#　XBRLの展開
#

for index, docData in EDINETdata2.iterrows():
    CName=index[0]
    DocType=index[1]
    Date=index[2]
    file_path_M=docData['XBRLパス']
    print('会社名:',CName)
    print('書類名:',DocType)
    print('日付:',Date)
    print('docID:',docData['docID'])
    print('Description:',docData['docDescription'])
    print('XBRLパス:',file_path_M)
    print(glob.glob(file_path_M+'/XBRL/PublicDoc/*.xbrl'))
    file_path=glob.glob(file_path_M+'/XBRL/PublicDoc/*.xbrl')[0]

    for 項目ラベル in 辞書Keyリスト:
        print('会社名:',CName)
        print('書類名:',DocType)
        print('日付:',Date)
        print(項目ラベル)
        print(項目辞書[項目ラベル])
        html=get_one_xbrl_data(file_path,
            key=項目辞書[項目ラベル],context_ref='FilingDateInstant')
        #print(html)
        if html is not None:
            soup = BeautifulSoup(html, "html5lib")
            text=soup.get_text()
        else:
            text=''
        #print(text)
        
        # textを改行ごとにリストに入れて、リスト内の要素の前後の空白を削除
        lines= [line.strip() for line in text.splitlines()]
    
        # リストの空白要素以外をすべて文字列に戻す
        E_text="\n".join(line for line in lines if line)
        print('\n＜＜＜テキスト＞＞＞',E_text)
        #print('============')
        EDINETdata2.at[index,項目ラベル]=E_text   
    #ループの終了
    #TMM.beep()

#処理の終了
beep3()

#
#  Dataframeを保存
#
fName='EDINET_S/Report2022/EDINETData.csv'
EDINETdata2.to_csv(fName,encoding='utf_8_sig')
#subprocess.Popen(['open','C:\Program Files\Microsoft Office\root\Office16\EXCEL.EXE',fName])

会社名: イチカワ
書類名: 有価証券報告書
日付: 2021-03-31
docID: S100LUEG
Description: 有価証券報告書－第97期(令和2年4月1日－令和3年3月31日)
XBRLパス: EDINET_S/Report2022/イチカワ/S100LUEG
['EDINET_S/Report2022/イチカワ/S100LUEG/XBRL/PublicDoc\\jpcrp030000-asr-001_E00572-000_2021-03-31_01_2021-06-29.xbrl']
会社名: イチカワ
書類名: 有価証券報告書
日付: 2021-03-31
研究開発
jpcrp_cor:ResearchAndDevelopmentActivitiesTextBlock

＜＜＜テキスト＞＞＞ ５ 【研究開発活動】当社グループの研究開発活動は、抄紙用具関連事業及び工業用事業ともに、優位化商品、新製品の開発及び技術開発を主体とし、また、環境に配慮したテーマをより多く取り上げて活動しております。抄紙用具関連事業の研究開発活動については、最新の市場動向や抄紙技術にお応えするため、新素材の応用、新樹脂、新加工法の開発及び基盤技術開発を主体に行っております。当連結会計年度につきましては、抄紙用ベルトにおいて、更なる機能向上を目指し、新製品の開発に取り組みました。なお、当社グループの研究開発費は、そのほとんどがセグメントに配分できない基礎研究であり、当連結会計年度中に支出した研究開発費の総額は332百万円です。
会社名: イチカワ
書類名: 有価証券報告書
日付: 2021-03-31
経営方針
jpcrp_cor:BusinessPolicyBusinessEnvironmentIssuesToAddressEtcTextBlock

＜＜＜テキスト＞＞＞ １ 【経営方針、経営環境及び対処すべき課題等】当社は、「事業は人なり而して人の和なり」「より良い品をより安くより多く」を社是とし、「株主重視」・「顧客重視」・「社員の生活向上」の理念に基づき、市場のニーズに的確に対応した高機能製品を提供する「抄紙用具の高度専門企業」として成長・発展することを目指して経営活動を展開しております。 当社グループを取り巻く経営環境は、国内の構造的な需要縮小やグローバル市場

In [23]:
file_path

'EDINET_S/Report2022/サンリオ/S100LOB9/XBRL/PublicDoc\\jpcrp030000-asr-001_E02655-000_2021-03-31_01_2021-06-25.xbrl'

In [21]:
##############################
#
#　保存したデータの読み込み
#
##############################

fName='EDINET_S/Report2022/EDINETData.csv'
print(fName)

EDINETdata2 = pd.read_csv(fName)
EDINETdata2 = EDINETdata2.set_index(['会社名','書類名','日付'])


EDINET_S/Report2022/EDINETData.csv


In [22]:
EDINETdata2

docID                    docDescription  \
会社名  書類名     日付                                                       
イチカワ 有価証券報告書 2021-03-31  S100LUEG  有価証券報告書－第97期(令和2年4月1日－令和3年3月31日)   
サンリオ 有価証券報告書 2021-03-31  S100LOB9  有価証券報告書－第61期(令和2年4月1日－令和3年3月31日)   

                                                    XBRLパス  \
会社名  書類名     日付                                              
イチカワ 有価証券報告書 2021-03-31  EDINET_S/Report2022/イチカワ/S100LUEG   
サンリオ 有価証券報告書 2021-03-31  EDINET_S/Report2022/サンリオ/S100LOB9   

                                                                      研究開発  \
会社名  書類名     日付                                                              
イチカワ 有価証券報告書 2021-03-31  ５ 【研究開発活動】当社グループの研究開発活動は、抄紙用具関連事業及び工業用事業ともに、優位...   
サンリオ 有価証券報告書 2021-03-31                              ５ 【研究開発活動】該当事項はありません。   

                                                                      経営方針  \
会社名  書類名     日付                                                              
イチカワ 有価証券報告書 2021-03-31  １ 【経営方針、経営環境及び対処すべき課題等】当社は、「事業は人なり而して人の和なり」「より...   
サンリオ 有価証券報告書 2021-03-31  １ 【経営方針、経営環境及び対処すべき課題等】文中の将来に関する事項は、当連結会計年度末現在...   

                                                                     事業リスク  \
会社名  書類名     日付                                                              
イチカワ 有価証券報告書 2021-03-31  ２ 【事業等のリスク】有価証券報告書に記載した事業の状況、経理の状況等に関する事項のうち、経...   
サンリオ 有価証券報告書 2021-03-31  ２ 【事業等のリスク】有価証券報告書に記載した事業の状況、経理の状況等に関する事項のうち、経...   

                                                                     ガバナンス  
会社名  書類名     日付                                                             
イチカワ 有価証券報告書 2021-03-31  (1) 【コーポレート・ガバナンスの概要】① コーポレート・ガバナンスに関する基本的な考え方...  
サンリオ 有価証券報告書 2021-03-31  (1) 【コーポレート・ガバナンスの概要】　　①　コーポレート・ガバナンスに関する基本的な考...